In [110]:
import numpy as np
import os
import pandas as pd
import csv
from matplotlib import pyplot as plt
import matplotlib.pyplot as pyplot
import PIL.Image as Image
from PIL import ImageOps
from scipy.cluster.hierarchy import dendrogram, linkage
from scipy.cluster.vq import vq, kmeans, whiten
import glob
import math
from sklearn.cluster import k_means
from sklearn.metrics import confusion_matrix

In [111]:
#[Larry's path] Brush_teeth_path = os.path.join(cur_dir,'data/Brush_teeth')
#[Titus's path] r'C:/Users/titus/Documents/CS498/hw4/HMP_Dataset/
cur_dir = os.path.dirname('__file__') #<-- absolute dir the script is in
Brush_teeth_path = os.path.join(cur_dir,'data/Brush_teeth')
Climb_stairs_MODEL_path = os.path.join(cur_dir,'data/Climb_stairs') 
Comb_hair_path = os.path.join(cur_dir,'data/Comb_hair')
Descend_stairs_path = os.path.join(cur_dir,'data/Descend_stairs')
Drink_glass_MODEL_path = os.path.join(cur_dir,'data/Drink_glass')
Eat_meat_path = os.path.join(cur_dir,'data/Eat_meat') 
Eat_soup_path = os.path.join(cur_dir,'data/Eat_soup')
Getup_bed_MODEL_path = os.path.join(cur_dir,'data/Getup_bed')
liedown_bed_path = os.path.join(cur_dir,'data/liedown_bed')
Pour_water_MODEL_path = os.path.join(cur_dir,'data/Pour_water')
Sitdown_chair_MODEL_path = os.path.join(cur_dir,'data/Sitdown_chair')
Standup_chair_MODEL_path = os.path.join(cur_dir,'data/Standup_chair')
Use_telephone_path = os.path.join(cur_dir,'data/Use_telephone')
Walk_MODEL_path = os.path.join(cur_dir,'data/Walk')

### Building Dataframes by Class

In [112]:
path_list = [Brush_teeth_path, Climb_stairs_MODEL_path, Comb_hair_path, Descend_stairs_path, Drink_glass_MODEL_path, Eat_meat_path, Eat_soup_path, Getup_bed_MODEL_path, liedown_bed_path, Pour_water_MODEL_path, Sitdown_chair_MODEL_path, Standup_chair_MODEL_path, Use_telephone_path, Walk_MODEL_path]
df_list = []
testDF_list = []
test_df = []
chunk_size = 32.0
first_c = 20
second_c = 12
seg_len = int(3 * chunk_size)
for path in path_list:
    allFiles = glob.glob(path + "/*.txt")
    a = False
    for file_ in allFiles: #take every sample of one action in one single df
        if a == False:
            frame = pd.read_csv(file_,sep=" ",index_col=None,  header=None)
            a = True
        else:
            df = pd.read_csv(file_,sep=" ",index_col=None,  header=None)
            frame = pd.concat([frame,df])
    newdf = frame.iloc[0:32,:]
    newnp = newdf.as_matrix()
    vq_df = pd.DataFrame(np.array(newnp).resize((1, seg_len)))
    
    #choseeing 20% to be the test data
    test_threshold = math.floor(len(frame.index)/chunk_size)

    set_vqDF = False
    
    for i in range (1, math.floor(len(frame.index)/chunk_size)):
        newdf = frame.iloc[32*i:32*(i)+31, ]
        newnp = newdf.as_matrix()
        npreshape = pd.DataFrame(np.resize(newnp,(1, seg_len)))
        if i < test_threshold:
            vq_df = pd.concat([vq_df,npreshape])
        else:
            if set_vqDF == False: 
                set_vqDF = True 
                test_df = pd.DataFrame(np.array(newnp).resize((1,seg_len)))
            else: 
                test_df = pd.concat([test_df,npreshape])
    df_list.append(vq_df)
    testDF_list.append(test_df)

### Combining Dataframes into one Train Dataframe

In [113]:
train_df = df_list[0]
for df in range(1,len(df_list)):
    train_df = pd.concat([train_df,df_list[df]])

### Building Dictionary

In [114]:
labels_list = ["Brush_teeth_path", "Climb_stairs_MODEL_path", "Comb_hair_path", "Descend_stairs_path", "Drink_glass_MODEL_path", "Eat_meat_path", "Eat_soup_path", "Getup_bed_MODEL_path", "liedown_bed_path", "Pour_water_MODEL_path", "Sitdown_chair_MODEL_path", "Standup_chair_MODEL_path", "Use_telephone_path", "Walk_MODEL_path"]
#890
init = True 
init_first_layer = [True]*first_c
first_layer = [pd.DataFrame(np.array(newnp).resize((1,seg_len)))]*first_c
whitene = whiten(train_df)
centroid, label, inertia = k_means(whitene.astype(float), first_c)
cent_from_first_layer = centroid
cent_from_second_layer = [0]*first_c
for i in range(label.shape[0]): 
    #row1 = (whitene.iloc[i]).as_matrix()
    row2 = pd.DataFrame(whitene[i].reshape((1,seg_len)))
    c = label[i]
    if init_first_layer[c] == True:
        init_first_layer[c] = False
        first_layer[c] =row2
    else:
        first_layer[c] = pd.concat([first_layer[c],row2])
histogram_vector = []
for i in range(first_c):
    cent_from_second_layer[i] = [0]*seg_len
    cur_cluster = first_layer[i] 
    whitene = whiten(cur_cluster)
    centroid, label, inertia = k_means(whitene.astype(float), second_c)
    for j in range(second_c): 
        cent_from_second_layer[i][j] = centroid[j]


### Breaking txt files into chunks ; spliting files into training and testing set

In [115]:
path_list = [Brush_teeth_path, Climb_stairs_MODEL_path, Comb_hair_path, Descend_stairs_path, Drink_glass_MODEL_path, Eat_meat_path, Eat_soup_path, Getup_bed_MODEL_path, liedown_bed_path, Pour_water_MODEL_path, Sitdown_chair_MODEL_path, Standup_chair_MODEL_path, Use_telephone_path, Walk_MODEL_path]
train_y = []
test_y = [] 
#path_list = [Brush_teeth_path] 
label = 0 
init_histogram = False
init_histogram2 = False


for path in path_list:
    allFiles = glob.glob(path + "/*.txt")
    threshold = int(len(allFiles)*0.8)
    trainFiles = allFiles[0:threshold]
    testFiles = allFiles[threshold:len(allFiles)]
    ###
    # Training data
    ###
    for file_ in trainFiles: #per file
        # make histogram per file 
        # find closest centroid
        frame = pd.read_csv(file_,sep=" ",index_col=None,  header=None)
        newdf = frame.iloc[0:32,:]
        newnp = newdf.as_matrix()
        vq_df = pd.DataFrame(np.array(newnp).resize((1,seg_len)))
        histogram = [0]*first_c*second_c
        for i in range (0, math.floor(len(frame.index)/chunk_size)):
            newdf = frame.iloc[32*i:32*(i)+31, ]
            newnp = newdf.as_matrix()
            npreshape = pd.DataFrame(np.resize(newnp,(1,seg_len)))
            ####
            # Find closest centroid 
            ####
            min_val = 99999
            min_cluster = 0
            for j in range(len(cent_from_first_layer)):
                val = np.sum(np.abs(cent_from_first_layer[j] - npreshape))
                val = sum(val)
                if val < min_val: 
                    min_val = val
                    min_cluster = j
            min_val2 = 99999
            min_cluster2 = 0
            for k in range(second_c):
                val = np.sum(np.abs(cent_from_second_layer[min_cluster][k] - npreshape))
                val = sum(val)
                if val < min_val2: 
                    min_val2 = val
                    min_cluster2 = k          
            histogram[second_c*min_cluster + min_cluster2] = histogram[second_c*min_cluster + min_cluster2] + 1 
        histogram = pd.DataFrame(np.array(histogram).reshape((1,first_c * second_c)))
        train_y.append(label)
        if init_histogram == False:
            total_train = histogram
            init_histogram = True
        else:
            total_train = pd.concat([total_train,histogram])
    
    ###
    # Testing data
    ###
    for file_ in testFiles: #per file
        # make histogram per file 
        # find closest centroid
        frame = pd.read_csv(file_,sep=" ",index_col=None,  header=None)
        newdf = frame.iloc[0:32,:]
        newnp = newdf.as_matrix()
        vq_df = pd.DataFrame(np.array(newnp).resize((1,seg_len)))
        histogram = [0]*first_c *second_c
        for i in range (0, math.floor(len(frame.index)/chunk_size)):
            newdf = frame.iloc[32*i:32*(i)+31, ]
            newnp = newdf.as_matrix()
            npreshape = pd.DataFrame(np.resize(newnp,(1,seg_len)))
            ####
            # Find closest centroid 
            ####
            min_val = 99999
            min_cluster = 0
            for j in range(len(cent_from_first_layer)):
                val = np.sum(np.abs(cent_from_first_layer[j] - npreshape))
                val = sum(val)
                if val < min_val: 
                    min_val = val
                    min_cluster = j
            min_val2 = 99999
            min_cluster2 = 0
            for k in range(12):
                val = np.sum(np.abs(cent_from_second_layer[min_cluster][k] - npreshape))
                val = sum(val)
                if val < min_val2: 
                    min_val2 = val
                    min_cluster2 = k          
            histogram[second_c *min_cluster + min_cluster2] = histogram[second_c*min_cluster + min_cluster2] + 1   
        histogram = pd.DataFrame(np.array(histogram).reshape((1,second_c * first_c)))
        test_y.append(label)
        if init_histogram2 == False:
            total_test = histogram
            init_histogram2 = True
        else:
            total_test = pd.concat([total_test,histogram])
            
    ###
    # Update label 
    ###
    label = label + 1


done
done
done
done
done
done
done
done
done
done
done
done
done
done


### Calculate Accuracy and P

In [116]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split

rf = RandomForestClassifier(n_estimators=15,max_depth=15 )
y_pred = rf.fit(total_train, train_y).predict(total_test)
result = np.sum(np.in1d(test_y,y_pred))/len(y_pred)
#test data 
print(sum([ x == y for x,y in zip(test_y, y_pred)])/len(y_pred))
print(confusion_matrix(test_y, y_pred))
#train data
y_pred = rf.fit(total_train, train_y).predict(total_train)
result = np.sum(np.in1d(train_y,y_pred))/len(y_pred)
print(sum([ x == y for x,y in zip(train_y, y_pred)])/len(y_pred))
print(confusion_matrix(train_y, y_pred))

0.6069364161849711
[[ 2  0  0  0  0  0  0  0  0  0  0  0  0  1]
 [ 0 16  0  3  0  0  0  0  0  0  0  1  0  1]
 [ 0  0  4  0  1  0  0  2  0  0  0  0  0  0]
 [ 0  2  0  7  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  2  0 12  0  0  2  0  3  1  0  0  0]
 [ 0  0  0  0  0  1  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  1  0  0  0  0  0  0  0]
 [ 0  0  2  0  2  0  0 11  1  1  0  4  0  0]
 [ 0  0  0  0  1  0  0  1  1  1  0  2  0  0]
 [ 0  0  0  0  8  0  0  2  0  9  1  0  0  0]
 [ 0  0  1  0  1  0  0  2  0  0 11  5  0  0]
 [ 0  0  0  0  4  0  0  1  0  0  7  9  0  0]
 [ 0  0  1  0  1  0  0  0  0  0  0  0  1  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0 20]]
0.7102102102102102
[[ 9  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0 75  0  4  0  0  0  2  0  0  0  0  0  0]
 [ 0  0 21  0  2  0  0  1  0  0  0  0  0  0]
 [ 0  1  0 32  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  4  0 48  0  0  6  0 20  1  0  1  0]
 [ 0  0  0  0  0  4  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  2  0  0  0  0  0  0  0]
 [ 0  0  0  0  6